In [177]:
import pandas as pd
import numpy as np
import time
import datetime

dimension_path = "../DimensionsPre/"
data_path = "../Data/"

In [178]:
Date_dimension = pd.read_csv(dimension_path+"Date_dimension/Date_dimension.csv")
Mobility_dimension = pd.read_csv(dimension_path+"Mobility_dimension/Mobility_dimension.csv")
Patient_dimension = pd.read_csv(dimension_path+"Patient_dimension/Patient_dimension.csv")
PHU_Location_dimension = pd.read_csv(dimension_path+"PHU_Location_dimension/PHU_Location_dimension.csv")
Special_Measures_dimension = pd.read_csv(dimension_path+"Special_Measures_dimension/Special_Measures_dimension.csv")
Weather_dimension = pd.read_csv(dimension_path+"Weather_dimension/Weather_dimension.csv")

In [179]:
cases = pd.read_csv(data_path+"IndividualCasesInOntario.csv")
# cases.head()

In [180]:
cases["Accurate_Episode_Date"] = pd.to_datetime(cases["Accurate_Episode_Date"]).dt.strftime('%Y-%m-%d')
cases["Case_Reported_Date"] = pd.to_datetime(cases["Case_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Test_Reported_Date"] = pd.to_datetime(cases["Test_Reported_Date"]).dt.strftime('%Y-%m-%d')
cases["Specimen_Date"] = pd.to_datetime(cases["Specimen_Date"]).dt.strftime('%Y-%m-%d')
cases = cases[(cases["Test_Reported_Date"] < "2021-03-01") & (cases["Test_Reported_Date"] >= "2020-11-01")]
# cases = cases[(cases["Reporting_PHU_City"] == "Toronto") | (cases["Reporting_PHU_City"] == "Ottawa")]
durhanCases = cases[(cases["Reporting_PHU"].map(lambda x: "durham" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "durham" in x.lower()))]
haltonCases = cases[(cases["Reporting_PHU"].map(lambda x: "halton" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "halton" in x.lower()))]
peelCases = cases[(cases["Reporting_PHU"].map(lambda x: "peel" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "peel" in x.lower()))]
yorkCases = cases[(cases["Reporting_PHU"].map(lambda x: "york" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "york" in x.lower()))]
ottawaCases = cases[(cases["Reporting_PHU"].map(lambda x: "ottawa" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "ottawa" in x.lower()))]
torontoCases = cases[(cases["Reporting_PHU"].map(lambda x: "toronto" in x.lower()))   |  (cases["Reporting_PHU_City"].map(lambda x: "toronto" in x.lower()))]
frames = [durhanCases, haltonCases, peelCases, yorkCases, ottawaCases, torontoCases]
cases = pd.concat(frames)
cases = cases.fillna({'Outbreak_Related':'Unknown'})[:67401:100]

In [181]:
cases[["Reporting_PHU","Reporting_PHU_City"]].drop_duplicates()

,Reporting_PHU,Reporting_PHU_City
2940,Durham Region Health Department,Whitby
5858,Halton Region Health Department,Oakville
3079,Peel Public Health,Mississauga
3424,York Region Public Health Services,Newmarket


In [182]:
reportingPHU2SubRegion2={
    "Durham Region Health Department":"Regional Municipality of Durham",
    "Halton Region Health Department":"Regional Municipality of Halton",
    "Peel Public Health":"Regional Municipality of Peel",
    "York Region Public Health Services":"Regional Municipality of York",
    "Ottawa Public Health":"Ottawa Division",
    "Toronto Public Health":"Toronto Division"
}

In [183]:
fact_table =  pd.DataFrame(columns=['Onset_date_key','Reported_date_key','Test_date_key', 'Specimen_date_key', 'Patient_key', "PHU_location_key","Special_measure_key","Mobility_key","Weather_key","Resolved","Unresolved","Fatal"])

In [184]:
fact_table.head()

,Onset_date_key,Reported_date_key,Test_date_key,Specimen_date_key,Patient_key,PHU_location_key,Special_measure_key,Mobility_key,Weather_key,Resolved,Unresolved,Fatal


In [186]:
#TO REMOVED:df.dropna(axis=0,how='any')
start = time.time()
cases = cases.dropna(axis=0,how='any')
# cases = cases.head()
rows = 0
Resolved,Unresolved,Fatal = 0,0,0
specialMeasureMismatch, mobilityMismatch, weatherMismatch = 0,0,0


for idx, row in cases.iterrows():
    rows += 1
    
    #not considering 2021 data
#     if (row["Case_Reported_Date"][:4]!="2020"):
#         continue
    
    if (rows%100 == 0):
        percentage = str(round((rows/len(cases))*100, 2)  )
        remaining = (time.time()-start)*((len(cases)-rows)/rows)
        print("%s/%s - %s%%  --about %s:%s left"%(str(rows),str(len(cases)),percentage,str(int(remaining//60)),str(int(remaining%60))),end = "\r")
    
    if (row["Outcome1"] == "Resolved"):
        Resolved+=1
    elif (row["Outcome1"] == "Not Resolved"):
        Unresolved+=1
    else:
        Fatal+=1

        
    # Special Measure
    try:
        specialMeasureId = Special_Measures_dimension[(Special_Measures_dimension["Reporting_PHU_id"]==row["Reporting_PHU_ID"])&(Special_Measures_dimension["Start-date"]<=row["Case_Reported_Date"])&(Special_Measures_dimension["End-date"]>=row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        specialMeasureMismatch+=1
        specialMeasureId = None
        
    # Mobility
    try:
        Mobility_dimensionId = Mobility_dimension[(Mobility_dimension["sub_region_2"] == reportingPHU2SubRegion2[row["Reporting_PHU"]])&(Mobility_dimension["date"]==row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        mobilityMismatch+=1
        Mobility_dimensionId = None
        
    # Weather
    try:
        Weather_dimensionId = Weather_dimension[(Weather_dimension["Station Name"]==row["Reporting_PHU_City"])&(Weather_dimension["Date/Time"]==row["Case_Reported_Date"])]["surrogate_key"].values[0]
    except IndexError as e:        
        weatherMismatch+=1
        Weather_dimensionId = None
        

    fact_row = [
    Date_dimension[Date_dimension["date"]==row["Accurate_Episode_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Case_Reported_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Test_Reported_Date"]]["surrogate_key"].values[0],
    Date_dimension[Date_dimension["date"]==row["Specimen_Date"]]["surrogate_key"].values[0],
    Patient_dimension[(Patient_dimension["Age_Group"]==row["Age_Group"])&(Patient_dimension["Client_Gender"]==row["Client_Gender"])&(Patient_dimension["Case_AcquisitionInfo"]==row["Case_AcquisitionInfo"])&(Patient_dimension["Outbreak_Related"]==row["Outbreak_Related"])]["surrogate_key"].values[0],
    PHU_Location_dimension[PHU_Location_dimension["Reporting_PHU_ID"]==row["Reporting_PHU_ID"]]["surrogate_key"].values[0],

    specialMeasureId,
    Mobility_dimensionId,
    Weather_dimensionId,
        
        
    Resolved,Unresolved,Fatal
    ]
    fact_table.loc[len(fact_table)] = fact_row

   
print("Finished Forming Fact Table Took %s:%s to Finish"%(int((time.time()-start)//60),int((time.time()-start)%60)))
print("Special Measure Mismatch: %s"%specialMeasureMismatch)
print("Mobility Mismatch: %s"%mobilityMismatch)
print("Weather Mismatch: %s"%weatherMismatch)
print("Total Cases: %s"%len(cases))
        
        
        
        
        

Finished Forming Fact Table Took 0:6 to Finish
Special Measure Mismatch: 5
Mobility Mismatch: 0
Weather Mismatch: 675
Total Cases: 675


In [ ]:
# fact_table.to_csv("Covid-19_Tracking_Fact_Table.csv",index=False)
fact_table

In [ ]:
fact_table[2990:2991]

In [ ]:
Date_dimension[Date_dimension["surrogate_key"]==356]["date"]

In [ ]:
Special_Measures_dimension[Special_Measures_dimension["surrogate_key"]==149]

In [ ]:
PHU_Location_dimension[PHU_Location_dimension["surrogate_key"]==0]

In [ ]:
fact_table.head()

In [ ]:
fact_table[2:3]

In [187]:
fact_table["Special_measure_key"].drop_duplicates()

0       15.0
1       17.0
2       13.0
3       16.0
14      14.0
50       NaN
88      43.0
89      40.0
90      41.0
95      42.0
159     97.0
165     98.0
171     96.0
362     99.0
626    165.0
628    168.0
629    166.0
630    164.0
635    167.0
Name: Special_measure_key, dtype: float64

In [ ]:
fact_table[166:167]

In [ ]:
fact_table["PHU_location_key"].drop_duplicates()